In [ ]:

import os
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

import warnings
warnings.filterwarnings('ignore')

embeddings = OpenAIEmbeddings(
    openai_api_key=os.environ["OPENAI_API_KEY"]
)

AZURE_SEARCH_SERVICE=os.environ["AZURE_SEARCH_SERVICE"]
AZURE_SEARCH_KEY=os.environ["AZURE_SEARCH_KEY"]

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=f"https://{AZURE_SEARCH_SERVICE}.search.windows.net",
    azure_search_key=AZURE_SEARCH_KEY,
    index_name="contoso-manuals",
    embedding_function=embeddings.embed_query)


In [ ]:
from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter

loader = DirectoryLoader(path="./manuals", glob="*.md", loader_cls=TextLoader)
documents = loader.load()
text_splitter = CharacterTextSplitter()
docs=text_splitter.split_documents(documents)
vector_store.add_documents(documents=docs)

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa = RetrievalQA.from_chain_type(OpenAI(), chain_type="stuff", retriever=vector_store.as_retriever())

In [ ]:
qa("what is the best tent for hard winds?")